Before Running the code -
+ Upload "dictionary.fullword" file
+ Upload a POS token csv file

#Installation

In [ ]:
%%capture
!pip install diffusers
!pip3 install ftfy
!pip3 install transformers
!pip3 install safetensors
!pip3 install accelerate

# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import re
import shutil
import json
import torch
from diffusers import StableDiffusionPipeline
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
from torch import autocast
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from safetensors.torch import save_file
from transformers import CLIPProcessor,CLIPModel

device=torch.device("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

#Find Nearest Neighbours in CLIP Token space
- dictionary.fullword file is required

In [ ]:
clipsrc="openai/clip-vit-large-patch14"
clip_processor=None
clip_model=None

In [ ]:
def init():
    global clip_processor
    global clip_model
    # Load the processor and model
    clip_processor = CLIPProcessor.from_pretrained(clipsrc)
    clip_model = CLIPModel.from_pretrained(clipsrc)
    clip_model = clip_model.to(device)

def standard_embed_calc(text):
    inputs = clip_processor(text=text, return_tensors="pt")
    inputs.to(device)
    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)
    embedding = text_features[0]
    return embedding

init()

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
with open("/content/dictionary.fullword","r") as f:
    tokendict = f.readlines()
    tokendict = [token.strip() for token in tokendict]  # Remove trailing newlines

In [ ]:
print("generate embeddings for each now")
count=1
all_embeddings = []
for word in tokendict:
    emb = standard_embed_calc(word)
    emb=emb.unsqueeze(0) # stupid matrix magic to make the cat work
    all_embeddings.append(emb)
    count+=1

generate embeddings for each now


In [ ]:
from safetensors.torch import save_file

embs = torch.cat(all_embeddings,dim=0)
embs = torch.squeeze(embs)
print("Shape of result = ",embs.shape)
# print("Saving all the things...")
# save_file({"embeddings": embs}, "embeddings.safetensors")

Shape of result =  torch.Size([32931, 768])


In [ ]:
def print_distances(targetemb, k):
    targetdistances = torch.cdist(targetemb.unsqueeze(0), embs, p=2)

    smallest_distances, smallest_indices = torch.topk(targetdistances[0], k, largest=True)  # k nearest neighbor

    smallest_distances=smallest_distances.tolist()
    smallest_indices=smallest_indices.tolist()

    tokenlist = []
    for d,i in zip(smallest_distances,smallest_indices):
        tokenlist.append(tokendict[i])
    return tokenlist

# Find closest tokens to targetword.
def find_closest(targetword, k):
    try:
        targetindex=tokendict.index(targetword)
        targetemb=embs[targetindex]
        tokenlist = print_distances(targetemb, k)
        return tokenlist
    except ValueError:
        print(targetword,"not found in cache")
        return []


#POS Identify

In [ ]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline
model_name = "QCRI/bert-base-multilingual-cased-pos-english"
pos_tokenizer = AutoTokenizer.from_pretrained(model_name)
pos_model = AutoModelForTokenClassification.from_pretrained(model_name)
pos_pipeline = TokenClassificationPipeline(model=pos_model, tokenizer=pos_tokenizer)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
skip_words = set(["man","woman","boy","girl","person","people","group","pair","couple","team","someone","individual","somebody","child","baby",
"squad","grouping","bunch","crowd","adult","guy","image","photo","picture","photograph","scene","herd","flock""pair"])

In [ ]:
def extract_proper_noun(sentence, poslist):
  prop_result = pos_pipeline(sentence)
  for result in prop_result:
    if result["entity"] in poslist and result["word"].lower().strip() not in skip_words and not result["word"].startswith("##"):
      return True

  return False

In [ ]:
def POS_extraction(targetlist, tagset, is_prop=False):
  temp_wrd = []
  if is_prop:
    for targ_word in targetlist:
      if extract_proper_noun(targ_word.capitalize(), tagset) == True:
        temp_wrd.append(targ_word)
  else:
    tokenized_text = targetlist
    for word, p_tag in pos_tag(tokenized_text):
      if p_tag in tagset and word.lower().strip() not in skip_words:
        temp_wrd.append(word.lower())

  return temp_wrd

#Fill-mask

In [ ]:
from transformers import pipeline
pipe = pipeline('fill-mask', model='bert-base-uncased')
tokenizer = pipe.tokenizer

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def fill_in_mask(text, target_word, targetlist, k, is_prop=False):
  text = text.replace(".", " .")
  words = text.replace("'", " '").split()
  index = words.index(target_word)
  masked_text = ' '.join([tokenizer.mask_token if i == index else w for i, w in enumerate(words)])
  #masked_text = text.replace(target_word, tokenizer.mask_token)
  if is_prop:
    if targetlist != None:
      tokenized_text = [word.lower() for word in targetlist]
  result = pipe(masked_text, targets=targetlist, top_k = k)
  return result

#Embedding

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"

embedder = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker = None)
embedder = embedder.to(device)
embedder.enable_attention_slicing()

embedder_tokenizer = embedder.tokenizer
embedder.text_encoder = embedder.text_encoder.float()
embedder_model = embedder.text_encoder
embedder_tokenizer.pad_token = embedder_tokenizer.eos_token

MAX_LENGTH = embedder_tokenizer.model_max_length

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
def get_embeddings(input_ids):
  return embedder_model.text_model.embeddings(input_ids)

# Perplexity Score

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
modelgpt2 = GPT2LMHeadModel.from_pretrained("gpt2-large").to(device)
tokenizergpt2 = GPT2TokenizerFast.from_pretrained("gpt2-large")

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def text_to_sentences(text):
    clean_text = text.replace('\n', ' ')
    return re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', clean_text)

def calculatePerplexity(text):
    encodings = tokenizergpt2("\n\n".join([text]), return_tensors="pt")
    max_length = modelgpt2.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = modelgpt2(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

    return ppl.item()

def calculatePerplexities(text):
    sentences = text_to_sentences(text)
    for sentence in sentences:
        perplexity = calculatePerplexity(sentence)
        score_dict = {"sentence": sentence, "perplexity": perplexity}
    return score_dict

#Find Target Prompts

In [ ]:
from matplotlib.colors import is_color_like

def get_antonyms(word):
  antonyms = []
  for syn in wordnet.synsets(word):
    for lemma in syn.lemmas():
      if lemma.antonyms():
        antonyms.append(lemma.antonyms()[0].name())
  return antonyms

def are_synonyms(word1, word2):
    synsets1 = wordnet.synsets(word1.lower().strip())
    synsets2 = wordnet.synsets(word2.lower().strip())
    return any(set(synset1.lemma_names()).intersection(synset2.lemma_names()) for synset1 in synsets1 for synset2 in synsets2)

def is_subtype(word_subtype, word_type):
    synsets_subtype = wordnet.synsets(word_subtype.lower().strip())
    synsets_type = wordnet.synsets(word_type.lower().strip())

    for syn_subtype in synsets_subtype:
        hypernyms = set([i for i in syn_subtype.closure(lambda s:s.hypernyms())])

        for syn_type in synsets_type:
            if syn_type in hypernyms:
                return True
    return False

def filter_candidate_words(target_word, candidate_list, poslist):
    # Remove target words, words containing target word and sub-words
    newlist = candidate_list.copy()
    for wrd in candidate_list:
      target_enc = tokenizer.tokenize(wrd)
      if target_word in wrd or (len(target_enc) > 1 or target_enc[0] == tokenizer.unk_token):
        newlist.remove(wrd)
        continue
      elif are_synonyms(wrd, target_word):
        newlist.remove(wrd)
        continue
      elif is_subtype(wrd, target_word) or is_subtype(target_word, wrd):
        newlist.remove(wrd)
    if poslist == ['JJ', 'JJR', 'JJS'] and is_color_like(target_word)==True:
      finallist = newlist.copy()
      for wrd in newlist:
        if wrd in target_word or is_color_like(wrd)==False:
          finallist.remove(wrd)
    else:
      finallist = newlist.copy()
    return finallist

In [ ]:
def find_candidate_words(input_text, target_word, poslist, clip_wrds, mask_wrds, df, is_prop=False):

  # find nearest words in clip token space
  targetlist = find_closest(target_word, clip_wrds)

  # get all words from the dataset
  for wrd in df['words']:
    targetlist.append(wrd.lower())

  # get antonyms for the word
  antonymlist = get_antonyms(target_word)
  for wrd in antonymlist:
    targetlist.append(wrd.lower())

  # get words from fill-mask model
  result = fill_in_mask(input_text, target_word, None, mask_wrds, is_prop)
  for res in result:
    token = res['token_str']
    targetlist.append(token)

  # keep only POS tagged words
  pos_targetlist = POS_extraction(targetlist, poslist, is_prop)

  # Remove target words, words containing target word, sub-words, synonyms, hypernyms
  new_targetlist = filter_candidate_words(target_word, pos_targetlist, poslist)
  return new_targetlist

In [ ]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-06)

def rank_candidate_prompts(input_text, target_word, candidate_words, is_prop):

    #get ranked candidate sentences
    top_ten = min(10, len(candidate_words))
    fill_mask_result = fill_in_mask(input_text, target_word, candidate_words, top_ten, is_prop)

    #calculate perplexity and select half population
    perplexity_dict =[]
    for fill_mask in fill_mask_result:
      target_text = fill_mask['sequence']

      if is_prop:
        tar_token = fill_mask['token_str']
        target_text = target_text.replace(tar_token, tar_token.capitalize())

      perplexity_dict.append(calculatePerplexities(target_text))
      # thisdict = {"sentence": target_text, "score":fill_mask['score']}
      # perplexity_dict.append(thisdict)

    top_five = min(5, top_ten) #len(candidate_words)//2
    top_five_prompts = (sorted(perplexity_dict, key=lambda x: x['perplexity'])[:top_five])
    top_five_prompts = perplexity_dict[:top_five]
    return top_five_prompts

def get_target_prompts(input_text, target_word, candidate_words, is_prop=False):

  candidate_prompts_dict = rank_candidate_prompts(input_text, target_word, candidate_words, is_prop)
  candidate_prompts_list = []
  for dic in candidate_prompts_dict:
   for key, val in dic.items():
      if key == "sentence":
        candidate_prompts_list.append(val)

  return candidate_prompts_list

# Main

In [ ]:
import pandas as pd
df = pd.read_csv("/content/PROPNOUN_token.csv")
df.head()

,captions,words,tag
0,two bananas and an apple are next to an Amazon...,Amazon,PROPER NOUN
1,a young boy wearing a Batman costume while rid...,Batman,PROPER NOUN
2,a hand holding a Blackberry phone taking a pic...,Blackberry,PROPER NOUN
3,city neon signs at dusk with a lot of those si...,Chinese,PROPER NOUN
4,living room decorated for Christmas with gifts...,Christmas,PROPER NOUN


In [ ]:
#Arguments

noun_poslist = ['NN', 'NNS']
prop_noun_poslist = ['NNP', 'NNPS']
verb_poslist = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
adverb_poslist = ['RB', 'RBR', 'RBS']
adjective_poslist = ['JJ', 'JJR', 'JJS']
numeric_poslist = ['CD']

no_of_wrds_from_clip = 100                # Number of words to find from CLIP Token Space
no_of_wrds_from_mask = 5                 # Number of words to find using Fill-Mask
poslist = prop_noun_poslist                  # Current POS tag
proper_noun_switch = True              # True when pos tag is PROPER NOUN
filename = "PROPNOUN_prompt_pairs.csv"   # filename for csv saving

In [ ]:
input_list, target_list = [], []

for idx in range(len(df)):
  input_text = df['captions'][idx]
  target_word = df['words'][idx]

  candidate_words = find_candidate_words(input_text.lower(), target_word.lower(), poslist, no_of_wrds_from_clip, no_of_wrds_from_mask, df, is_prop=proper_noun_switch)
  selected_prompts = get_target_prompts(input_text.lower(), target_word.lower(), candidate_words, is_prop=proper_noun_switch)

  print("Input Prompt: ",input_text)
  for i, sent in enumerate(selected_prompts):
    print("Target Prompt {}: {}".format(i, sent))
    input_list.append(input_text)
    target_list.append(sent)
  print("-"*50)

print("Complete!")

new_df = pd.DataFrame()
new_df['input_prompt'] = input_list
new_df['target_prompt'] = target_list
new_df.to_csv(filename, index=False)

Input Prompt:  two bananas and an apple are next to an Amazon box.
Target Prompt 0: two bananas and an Apple are next to an Apple box.
Target Prompt 1: two bananas and an apple are next to an Christmas box.
Target Prompt 2: two bananas and an apple are next to an Chinese box.
Target Prompt 3: two bananas and an apple are next to an Egypt box.
Target Prompt 4: two bananas and an apple are next to an Santa box.
--------------------------------------------------
Input Prompt:  a young boy wearing a Batman costume while riding a skateboard.
Target Prompt 0: a young boy wearing a Halloween costume while riding a skateboard.
Target Prompt 1: a young boy wearing a Superman costume while riding a skateboard.
Target Prompt 2: a young boy wearing a Santa costume while riding a skateboard.
Target Prompt 3: a young boy wearing a Christmas costume while riding a skateboard.
Target Prompt 4: a young boy wearing a Chinese costume while riding a skateboard.
--------------------------------------------

In [ ]:
are_synonyms("paint", "draw")

False